In [1]:
from shelve import open as open_shelve
from matplotlib import pyplot as plt
%matplotlib qt
from kitt_net import FeedForwardNet
from kitt_learning import Backpropagation
from sklearn.metrics import confusion_matrix, accuracy_score
from numpy import newaxis, zeros
from cPickle import dump as dump_cpickle, load as load_cpickle

In [2]:
dataset = open_shelve('../examples/train/dataset_train.ds')

In [3]:
how = {1: 'b.', -1: 'r.'}
for x, y in zip(dataset['x'], dataset['y']):
    plt.plot(x[0], x[1], how[y])
plt.grid()
plt.show()

In [16]:
net = FeedForwardNet(hidden=[5], tf_name='Sigmoid')

In [17]:
n_epoch = 10
learning_rate = 0.03
net.fit(x=dataset['x'], y=dataset['y'], x_val=dataset['x_val'], y_val=dataset['y_val'],
       learning_rate=learning_rate, n_epoch=n_epoch, batch_size=16, nd_der=True)


--------------------------------------------------------------------
-- Network initialized.
	% problem dimension: 7
	% number of classes: 2
	% class labels: ['east', 'west']
	% net structure: [7, 5, 2]
	% net transfer function: Sigmoid

--------------------------------------------------------------------
-- Learning has started...
	% problem dimension: 7
	% number of training samples: 1600
	% number of validation samples: 200
	% learning rate: 0.03
	% mini-batch size: 16
	% maximum number of epochs (t.c.): 10
	% maximum number of stable epochs (t.c.): inf
	% required accuracy (t.c.): inf
	% required error (t.c.): -inf


epoch   on training data      on validation data       epoch time          
-------------------------------------------------------------------
 1	  1.00/0.0432		1.00/0.0430		0.2537 s
 2	  1.00/0.0198		1.00/0.0196		0.2568 s
 3	  1.00/0.0110		1.00/0.0110		0.2464 s
 4	  1.00/0.0070		1.00/0.0070		0.3651 s
 5	  1.00/0.0049		1.00/0.0049		0.2510 s
 6	  1.00/0.0037		1.00/0.0

In [6]:
print net.w[0]
print '---------------'
print 'MAGNITUDE'
print abs(net.w)[0]

print 'KITT'
print abs(net.w-net.w_init)[0]

print 'KARNIN'
changes_ = [zeros(shape=w_i.shape) for w_i in net.w]
for ep_i in range(net.dw_i):
    for l_i in range(len(changes_)):
        den = (net.learning.kw['learning_rate']*(net.w[l_i]-net.w_init[l_i]))
        changes_[l_i] += net.dw_container[l_i][ep_i]**2*(net.w[l_i]/den)
print changes_[0]
        
print 'OBD'                    
print net.saliency[0]            

[[-0.62049887  0.19854937  0.05709136  0.59903029  0.35762892 -1.77111721
  -0.44660612]
 [ 1.68744513  1.73052508  2.18426972 -2.48605549  0.87730707 -1.17092054
   0.03726674]
 [ 0.77312382  0.82975343  1.26090305 -0.30481048  0.22644409 -0.86975571
   0.35669254]
 [ 2.0730543   0.89765229  0.83481316 -0.37020849 -0.1332012  -1.99967051
   1.32812084]
 [-2.29316897 -2.3454165  -2.41072633  0.86344918 -0.45561463  2.00904137
  -0.60571604]]
---------------
MAGNITUDE
[[ 0.62049887  0.19854937  0.05709136  0.59903029  0.35762892  1.77111721
   0.44660612]
 [ 1.68744513  1.73052508  2.18426972  2.48605549  0.87730707  1.17092054
   0.03726674]
 [ 0.77312382  0.82975343  1.26090305  0.30481048  0.22644409  0.86975571
   0.35669254]
 [ 2.0730543   0.89765229  0.83481316  0.37020849  0.1332012   1.99967051
   1.32812084]
 [ 2.29316897  2.3454165   2.41072633  0.86344918  0.45561463  2.00904137
   0.60571604]]
KITT
[[ 0.59817213  0.64615602  0.85461298  0.15748485  0.441028    0.5041532
   0

In [18]:
net.prune(req_acc=1.0, n_epoch=10, measure='karnin', retrain=True)


--------------------------------------------------------------------
-- Pruning has started...
	% net initial structure: [7, 5, 2]
	% net initial number of synapses (w, b): (45, 7)
	% min required accuracy (s.c.): 1.0
	% max required error (s.c.): 0.0
	% maximum number of re-training epochs (gu.c.): 10
	% number of stable iterations (gu.c.): 10


step    trying to cut     structure           left synapses (w/b)    retrained      next level     step time      
--------------------------------------------------------------------------------------------------------------
0       0                 [7, 5, 2]           (45, 7)                None           75             None  
1       34                [4, 1, 2]           (5, 3)                 yes            75             1.6722 s
2       4                 [1, 0, 2]           (0, 2)                 no             50             1.6202 s
3       3                 [1, 1, 2]           (2, 3)                 no             35             1.5

In [6]:
labels = [label for label in net.labels if label in dataset['y_val']]
y_pred = [net.predict(x)[0][0] for x, y in net.v_data]
print 'Acc:', accuracy_score(y_true=dataset['y_val'], y_pred=y_pred)
cm = confusion_matrix(y_true=dataset['y_val'], y_pred=y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, newaxis]
plt.imshow(cm, aspect='auto', interpolation='none', vmin=0, vmax=1)
plt.xticks(range(len(labels)), labels)
plt.yticks(range(len(labels)), labels)
plt.grid()
plt.colorbar()
plt.show()
print cm
for i in range(cm.shape[0]):
    print i, net.labels[i], sorted([(net.labels[j], round(cm[i, j], 2)) for j in range(cm.shape[1])], key=lambda x:x[1], reverse=True)[:5]


Acc: 1.0
[[ 1.  0.]
 [ 0.  1.]]
0 east [('east', 1.0), ('west', 0.0)]
1 west [('west', 1.0), ('east', 0.0)]


In [9]:
print net.used_features
print net.b

[(0, 0), (1, 1)]
[array([[-2.84159054],
       [ 1.90158142]]), array([[ 1.37822422],
       [ 2.95678191]])]
